In [1]:
from __future__ import division, print_function
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
plt.rcParams['image.cmap'] = 'gist_earth'
np.random.seed(98765)

In [2]:
import tensorflow as tf
tf.test.is_gpu_available()
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
tf.test.gpu_device_name()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16020993712792677032
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15862461236
locality {
  bus_id: 1
  links {
  }
}
incarnation: 7893464480001577617
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:0f:00.0, compute capability: 6.0"
]


'/device:GPU:0'

In [3]:
import sys
# Add the tf_unet folder path to the sys.path list
sys.path.append('/data3/floraxue/cs282a/SegmentNet/')


In [4]:
from tf_unet import image_gen
from tf_unet import unet
from tf_unet import util, image_util

In [5]:
#Data input
#put your data here
path_images = "/data3/floraxue/cs282a/data/dataset_1/trainval/C_phase/"
generator = image_util.ImageDataProvider(path_images+"*.tif")
#TODO: need to enhance the borders and rethink the matrix vs bkg 

Number of files used: 136900
Number of channels: 1
Number of classes: 2


In [6]:
net = unet.Unet(channels=1, n_class=2, layers=4, features_root=32)

2019-05-02 23:38:46,776 Layers 4, features 32, filter size 3x3, pool size: 2x2


In [9]:
trainer = unet.Trainer(net, batch_size=32, verification_batch_size = 320, 
                       optimizer="adam", opt_kwargs=dict(learning_rate = 0.001))

In [ ]:
path = trainer.train(generator, "./unet_C_adam", training_iters=4300, epochs=3, display_step=1000)

2019-05-02 23:39:30,683 Removing '/data3/floraxue/cs282a/SegmentNet/demo/prediction'
2019-05-02 23:39:30,688 Removing '/data3/floraxue/cs282a/SegmentNet/demo/unet_C_adam'
2019-05-02 23:39:30,718 Allocating '/data3/floraxue/cs282a/SegmentNet/demo/prediction'
2019-05-02 23:39:36,167 Verification error= 37.5%, loss= 0.6868
2019-05-02 23:39:39,991 Start optimization
2019-05-02 23:39:43,623 Iter 0, Minibatch Loss= 0.6769, Training Accuracy= 0.6059, Minibatch error= 39.4%
2019-05-02 23:48:48,093 Iter 1000, Minibatch Loss= 0.3324, Training Accuracy= 0.8530, Minibatch error= 14.7%
2019-05-02 23:57:53,540 Iter 2000, Minibatch Loss= 0.2037, Training Accuracy= 0.9147, Minibatch error= 8.5%
2019-05-03 00:06:58,964 Iter 3000, Minibatch Loss= 0.2977, Training Accuracy= 0.8606, Minibatch error= 13.9%
2019-05-03 00:16:03,044 Iter 4000, Minibatch Loss= 0.2010, Training Accuracy= 0.9113, Minibatch error= 8.9%
2019-05-03 00:18:45,456 Epoch 0, Average loss: 0.2695, learning rate: 0.0010
2019-05-03 00:18:4

In [ ]:
from skimage import io
x_test = io.imread(path_images+'0000.tif')
y_test = io.imread(path_images+'0000_mask.tif')

In [ ]:
fig, ax = plt.subplots(1,2, sharey=True, figsize=(10,5))
ax[0].imshow(x_test, aspect="auto")
ax[1].imshow(y_test, aspect="auto")

In [ ]:
x_test.shape

In [ ]:
#x_test, y_test = generator(1)
h,w = x_test.shape
x = x_test.reshape(1,h,w,1)
prediction = net.predict("./unet_trained/model.ckpt", x)

In [ ]:
fig, ax = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12,5))
ax[0].imshow(x_test, aspect="auto")
ax[1].imshow(y_test, aspect="auto")
mask = prediction[0,...,1] > 0.1
ax[2].imshow(mask, aspect="auto")
ax[0].set_title("Input")
ax[1].set_title("Ground truth")
ax[2].set_title("Prediction")

In [ ]:
fig.tight_layout()
fig.savefig("../docs/toy_problem.png")